In [5]:
import numpy as np
import pandas as pd

# Cargo la base limpia que salió del preprocesamiento del proyecto
df = pd.read_csv("df_cleaned.csv")

# Me quedo solo con precios positivos
df = df[df["price"] > 0].copy()

# Columnas numéricas (incluye dummies y variables binarias)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Quito el price original del set de X
numeric_cols.remove("price")

print("Número de columnas numéricas usadas como X:", len(numeric_cols))

# Split train / test 80% - 20%
train = df.sample(frac=0.8, random_state=100)
test = df.drop(train.index)

print("Tamaño train:", train.shape)
print("Tamaño test :", test.shape)

# X de entrenamiento y prueba
train_X = train[numeric_cols].copy()
test_X = test[numeric_cols].copy()

# y en log(precio), pero SIN guardar la columna en el DataFrame
train_y = np.log(train["price"].values)
test_y = np.log(test["price"].values)

print("Shapes finales X / y:")
print("train_X:", train_X.shape, "| train_y:", train_y.shape)
print("test_X :", test_X.shape, "| test_y :", test_y.shape)


Número de columnas numéricas usadas como X: 98
Tamaño train: (16662, 105)
Tamaño test : (4165, 105)
Shapes finales X / y:
train_X: (16662, 98) | train_y: (16662,)
test_X : (4165, 98) | test_y : (4165,)
